In [21]:
using Pkg
Pkg.activate("CT")
using CT
include("global_adder_passthrough.jl")

# benchmark adder_MPO
using ITensors
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO, add1, power_mpo
using Random

L = 40
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 1
p_ctrl = 1.0
p_proj = 0.0
_maxdim = 50
_cutoff = 1e-15
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
println(phy_list)
println(ram_phy)
println(phy_ram)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
# initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);
# println(initial_state)
shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
# initial_state = productMPS(qubit_site, [1,2,1,2,2,1,2,1,2,1,2,1,1,1,2,2])
initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);

# initialize haining adder mpo
add1_mpo=MPO(add1(i1,L,phy_ram,phy_list),qubit_site)
add1_6,add1_3=power_mpo(add1_mpo,[div(2^L,6)+1,div(2^L,3)])

# initialize tao adder mpo
carry_links, T_vec, id_vec, gate_vec = initialize_links(L, qubit_site, shift_1_3_bits, ram_phy);
initial_state_1 = copy(initial_state)
initial_state_2 = copy(initial_state)

  Activating project at `~/CT_MPS_mini/CT`


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
[1, 40, 2, 39, 3, 38, 4, 37, 5, 36, 6, 35, 7, 34, 8, 33, 9, 32, 10, 31, 11, 30, 12, 29, 13, 28, 14, 27, 15, 26, 16, 25, 17, 24, 18, 23, 19, 22, 20, 21]
[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 40, 38, 36, 34, 32, 30, 28, 26, 24, 22, 20, 18, 16, 14, 12, 10, 8, 6, 4, 2]


MPS
[1] ((dim=2|id=452|"Qubit,Site,n=1"), (dim=50|id=472|"Link,l=1"))
[2] ((dim=50|id=472|"Link,l=1"), (dim=2|id=53|"Qubit,Site,n=2"), (dim=50|id=958|"Link,l=2"))
[3] ((dim=50|id=958|"Link,l=2"), (dim=2|id=894|"Qubit,Site,n=3"), (dim=50|id=682|"Link,l=3"))
[4] ((dim=50|id=682|"Link,l=3"), (dim=2|id=522|"Qubit,Site,n=4"), (dim=50|id=497|"Link,l=4"))
[5] ((dim=50|id=497|"Link,l=4"), (dim=2|id=294|"Qubit,Site,n=5"), (dim=50|id=859|"Link,l=5"))
[6] ((dim=50|id=859|"Link,l=5"), (dim=2|id=642|"Qubit,Site,n=6"), (dim=50|id=582|"Link,l=6"))
[7] ((dim=50|id=582|"Link,l=6"), (dim=2|id=621|"Qubit,Site,n=7"), (dim=50|id=209|"Link,l=7"))
[8] ((dim=50|id=209|"Link,l=7"), (dim=2|id=220|"Qubit,Site,n=8"), (dim=50|id=167|"Link,l=8"))
[9] ((dim=50|id=167|"Link,l=8"), (dim=2|id=942|"Qubit,Site,n=9"), (dim=50|id=357|"Link,l=9"))
[10] ((dim=50|id=357|"Link,l=9"), (dim=2|id=741|"Qubit,Site,n=10"), (dim=50|id=124|"Link,l=10"))
[11] ((dim=50|id=124|"Link,l=10"), (dim=2|id=985|"Qubit,Site,n=11"), (dim=50|id=47

In [22]:
using Profile

In [23]:
# ProfileView removed due to glibc compatibility issues
# Use built-in Julia profiling instead:
@time global_adder(initial_state_2, carry_links, T_vec, id_vec, gate_vec, qubit_site, shift_1_3_bits, ram_phy)


 14.305471 seconds (104.12 k allocations: 4.736 GiB, 1.35% gc time, 1.79% compilation time)


MPS
[1] ((dim=2|id=452|"Qubit,Site,n=1"), (dim=4|id=116|"Link,l=1"))
[2] ((dim=2|id=53|"Qubit,Site,n=2"), (dim=4|id=116|"Link,l=1"), (dim=16|id=110|"Link,l=2"))
[3] ((dim=16|id=110|"Link,l=2"), (dim=2|id=894|"Qubit,Site,n=3"), (dim=32|id=357|"Link,l=3"))
[4] ((dim=32|id=357|"Link,l=3"), (dim=2|id=522|"Qubit,Site,n=4"), (dim=64|id=445|"Link,l=4"))
[5] ((dim=64|id=445|"Link,l=4"), (dim=2|id=294|"Qubit,Site,n=5"), (dim=128|id=235|"Link,l=5"))
[6] ((dim=128|id=235|"Link,l=5"), (dim=2|id=642|"Qubit,Site,n=6"), (dim=256|id=445|"Link,l=6"))
[7] ((dim=256|id=445|"Link,l=6"), (dim=2|id=621|"Qubit,Site,n=7"), (dim=512|id=571|"Link,l=7"))
[8] ((dim=512|id=571|"Link,l=7"), (dim=2|id=220|"Qubit,Site,n=8"), (dim=800|id=714|"Link,l=8"))
[9] ((dim=800|id=714|"Link,l=8"), (dim=2|id=942|"Qubit,Site,n=9"), (dim=800|id=525|"Link,l=9"))
[10] ((dim=800|id=525|"Link,l=9"), (dim=2|id=741|"Qubit,Site,n=10"), (dim=800|id=491|"Link,l=10"))
[11] ((dim=800|id=491|"Link,l=10"), (dim=2|id=985|"Qubit,Site,n=11"), (di

In [24]:
# Profile.print(format=:flat, sortedby=:count)

# # For now, just run the function without profiling:
# # global_adder(initial_state_2, carry_links, T_vec, id_vec, gate_vec, qubit_site, shift_1_3_bits, ram_phy)